# 2.5 Análisis Temporal y Comparativo de Estados Financieros

**Finanzas y Control Empresario - UTN La Plata**  
**Carrera:** Ingeniería Industrial  
**Profesor:** Nicolás Félix  
**Unidad 2:** Análisis de Estados Financieros

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/nfelix23/finanzas-control-empresario-utn/blob/main/notebooks/unidad_2/2.5_analisis_temporal_comparativo.ipynb)

---

## 🎯 Objetivos de Aprendizaje

Al finalizar este notebook, los estudiantes serán capaces de:

1. **Realizar** análisis horizontal de evolución temporal de ratios
2. **Implementar** análisis vertical de estructura financiera
3. **Ejecutar** benchmarking sectorial avanzado
4. **Identificar** tendencias y patrones en el desempeño financiero
5. **Desarrollar** análisis predictivo y de escenarios
6. **Crear** dashboards interactivos para monitoreo continuo

---

## 📚 Marco Teórico

### 📈 Análisis Horizontal (Temporal)

El **análisis horizontal** examina la evolución de los estados financieros a través del tiempo, permitiendo identificar:

- **Tendencias de crecimiento** en ventas, activos y resultados
- **Patrones estacionales** y cíclicos
- **Puntos de inflexión** en el desempeño
- **Impacto de decisiones estratégicas**

#### Fórmulas Clave:

$$\text{Variación Absoluta} = \text{Valor}_t - \text{Valor}_{t-1}$$

$$\text{Variación Relativa} = \frac{\text{Valor}_t - \text{Valor}_{t-1}}{\text{Valor}_{t-1}} \times 100$$

$$\text{CAGR} = \left(\frac{\text{Valor Final}}{\text{Valor Inicial}}\right)^{\frac{1}{n}} - 1$$

### 📊 Análisis Vertical (Estructural)

El **análisis vertical** expresa cada partida como porcentaje de una cifra base:

- **Balance**: Cada cuenta como % del Activo Total
- **Estado de Resultados**: Cada partida como % de las Ventas
- **Permite comparaciones** independientes del tamaño

#### Fórmulas:

$$\text{\% Vertical} = \frac{\text{Partida Individual}}{\text{Cifra Base}} \times 100$$

### 🏆 Benchmarking Sectorial

Comparación sistemática con:
- **Competidores directos**
- **Líderes sectoriales**
- **Promedios de industria**
- **Best practices internacionales**

### 📈 Análisis de Tendencias

Identificación de patrones mediante:
- **Regresión lineal**
- **Medias móviles**
- **Análisis estacional**
- **Detección de outliers**

---

In [ ]:
# Configuración del entorno
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
from datetime import datetime, timedelta
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from scipy import stats

# Configuración de visualización
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")
warnings.filterwarnings('ignore')

pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', '{:.2f}'.format)

print("📊 Entorno configurado correctamente")
print(f"📅 Fecha de ejecución: {datetime.now().strftime('%d/%m/%Y %H:%M')}")
print("\n🏛️ UTN La Plata - Finanzas y Control Empresario")
print("📖 Unidad 2.5: Análisis Temporal y Comparativo")

## 💼 Caso de Estudio: Series Temporales Empresariales

Crearemos **series temporales realistas** para nuestras empresas argentinas, simulando 5 años de evolución con diferentes estrategias y contextos económicos:

In [ ]:
# Generar datos temporales realistas para análisis
def generar_serie_temporal_empresa(empresa_base, años=5, volatilidad=0.1):
    """
    Genera serie temporal realista basada en datos base de la empresa
    """
    
    periodos = []
    base_year = 2020
    
    # Factores macroeconómicos argentinos simulados
    macro_factors = {
        2020: {'gdp_growth': -9.9, 'inflation': 42.0, 'usd_cop': 0.15},  # Pandemia
        2021: {'gdp_growth': 10.4, 'inflation': 48.4, 'usd_cop': 0.12},  # Recuperación
        2022: {'gdp_growth': 5.0, 'inflation': 72.4, 'usd_cop': 0.08},   # Inflación alta
        2023: {'gdp_growth': -1.6, 'inflation': 133.0, 'usd_cop': 0.05}, # Crisis
        2024: {'gdp_growth': 2.8, 'inflation': 89.2, 'usd_cop': 0.07}    # Estabilización
    }
    
    for año in range(base_year, base_year + años):
        macro = macro_factors[año]
        
        # Ajustar según sector y contexto macro
        if empresa_base['sector'] == 'Energético':
            # Sector energético: impacto de precios commodities y regulación
            growth_factor = 1 + (macro['gdp_growth'] * 0.8 + np.random.normal(0.03, volatilidad))
            margin_impact = 1 + (macro['usd_cop'] * 2 + np.random.normal(0, volatilidad/2))
            
        elif empresa_base['sector'] == 'Financiero':
            # Sector financiero: beneficiado por alta inflación en spreads
            growth_factor = 1 + (macro['inflation']/100 * 0.3 + np.random.normal(0.05, volatilidad))
            margin_impact = 1 + (macro['inflation']/100 * 0.2 + np.random.normal(0, volatilidad/3))
            
        elif empresa_base['sector'] == 'Utilities':
            # Utilities: tarifas reguladas, crecimiento estable
            growth_factor = 1 + (macro['gdp_growth'] * 0.4 + np.random.normal(0.02, volatilidad/2))
            margin_impact = 1 + np.random.normal(0, volatilidad/4)
            
        elif empresa_base['sector'] == 'Alimentos':
            # Alimentos: defensivo pero impactado por costos
            growth_factor = 1 + (macro['gdp_growth'] * 0.6 + np.random.normal(0.04, volatilidad))
            margin_impact = 1 - (macro['inflation']/100 * 0.1 + np.random.normal(0, volatilidad/3))
            
        else:  # Telecomunicaciones
            # Telecom: crecimiento digital pero presión regulatoria
            growth_factor = 1 + (macro['gdp_growth'] * 0.5 + np.random.normal(0.06, volatilidad))
            margin_impact = 1 + np.random.normal(0.01, volatilidad/3)
        
        # Asegurar valores no negativos
        growth_factor = max(0.7, growth_factor)
        margin_impact = max(0.8, margin_impact)
        
        if año == base_year:
            # Año base
            ventas = empresa_base['ventas']
            activo_total = empresa_base['activo_total']
            patrimonio_neto = empresa_base['patrimonio_neto']
            resultado_neto = empresa_base['resultado_neto']
        else:
            # Años siguientes con evolución
            periodo_anterior = periodos[-1]
            
            ventas = periodo_anterior['ventas'] * growth_factor
            activo_total = periodo_anterior['activo_total'] * (growth_factor ** 0.7)  # Menor crecimiento activos
            
            # Patrimonio evoluciona con retención de utilidades
            payout_ratio = 0.3  # 30% dividendos
            patrimonio_neto = (periodo_anterior['patrimonio_neto'] + 
                             periodo_anterior['resultado_neto'] * (1 - payout_ratio))
            
            # Resultado neto con variabilidad
            base_margin = empresa_base['resultado_neto'] / empresa_base['ventas']
            new_margin = base_margin * margin_impact
            resultado_neto = ventas * new_margin
        
        periodos.append({
            'año': año,
            'ventas': ventas,
            'activo_total': activo_total,
            'patrimonio_neto': patrimonio_neto,
            'pasivo_total': activo_total - patrimonio_neto,
            'resultado_neto': resultado_neto,
            'gdp_growth': macro['gdp_growth'],
            'inflation': macro['inflation']
        })
    
    return pd.DataFrame(periodos)

# Datos base de empresas (del notebook anterior)
empresas_base = {
    'YPF': {
        'nombre': 'YPF S.A.',
        'sector': 'Energético',
        'ventas': 2456700,
        'activo_total': 3714300,
        'patrimonio_neto': 934900,
        'resultado_neto': 231770
    },
    'MACRO': {
        'nombre': 'Banco Macro S.A.',
        'sector': 'Financiero',
        'ventas': 456700,
        'activo_total': 1567800,
        'patrimonio_neto': 333300,
        'resultado_neto': 94220
    },
    'PAMPA': {
        'nombre': 'Pampa Energía S.A.',
        'sector': 'Utilities',
        'ventas': 1234500,
        'activo_total': 2345600,
        'patrimonio_neto': 777800,
        'resultado_neto': 125230
    },
    'ARCOR': {
        'nombre': 'Arcor S.A.I.C.',
        'sector': 'Alimentos',
        'ventas': 1678900,
        'activo_total': 1456700,
        'patrimonio_neto': 589300,
        'resultado_neto': 132580
    },
    'TELECOM': {
        'nombre': 'Telecom Argentina S.A.',
        'sector': 'Telecomunicaciones',
        'ventas': 987600,
        'activo_total': 1234500,
        'patrimonio_neto': 456700,
        'resultado_neto': 101920
    }
}

# Generar series temporales para todas las empresas
series_temporales = {}
for codigo, empresa in empresas_base.items():
    np.random.seed(42 + hash(codigo) % 100)  # Semilla reproducible pero única por empresa
    series_temporales[codigo] = generar_serie_temporal_empresa(empresa)
    series_temporales[codigo]['empresa'] = empresa['nombre']
    series_temporales[codigo]['sector'] = empresa['sector']

print("📊 Series temporales generadas exitosamente")
print(f"📅 Período de análisis: 2020-2024 (5 años)")
print(f"🏢 Empresas incluidas: {len(series_temporales)}")

# Mostrar ejemplo de datos generados
print("\n📈 Ejemplo - YPF S.A. (Evolución de Ventas):")
ypf_data = series_temporales['YPF']
for _, row in ypf_data.iterrows():
    print(f"  {row['año']}: ${row['ventas']:,.0f} MM (Crecimiento PIB: {row['gdp_growth']:+.1f}%)")

print("\n✅ Datos listos para análisis temporal y comparativo")

## 🎯 Conceptos Clave Aprendidos

### ✅ Checklist de Comprensión

Marca los conceptos que has dominado:

**Análisis Horizontal:**
- [ ] **Variaciones Absolutas y Relativas**: Cálculo año a año
- [ ] **CAGR (Tasa de Crecimiento Anual Compuesta)**: Métrica de crecimiento
- [ ] **Identificación de Tendencias**: Patrones de evolución temporal
- [ ] **Análisis de Puntos de Inflexión**: Momentos de cambio estratégico

**Análisis Vertical:**
- [ ] **Estructura Porcentual**: Cada partida como % de la base
- [ ] **Análisis de Márgenes**: Evolución de eficiencia operativa
- [ ] **Identificación de Drivers de Costos**: Principales componentes
- [ ] **Comparación Intersectorial**: Estructuras por industria

**Benchmarking:**
- [ ] **Benchmarks Sectoriales**: Comparación con pares
- [ ] **Posicionamiento Relativo**: Líder vs seguidor
- [ ] **Identificación de Best Practices**: Mejores prácticas
- [ ] **Gap Analysis**: Brechas de desempeño

**Análisis Predictivo:**
- [ ] **Análisis de Tendencias**: Regresión y proyecciones
- [ ] **Correlaciones Macroeconómicas**: Sensibilidad a factores externos
- [ ] **Análisis de Escenarios**: Proyecciones bajo diferentes contextos
- [ ] **Gestión de Incertidumbre**: Rangos de confianza

### 📊 Métricas Temporales Fundamentales

| Métrica | Fórmula | Interpretación |
|---------|---------|----------------|
| **Variación Relativa** | (Valor_t - Valor_t-1) / Valor_t-1 | Crecimiento período a período |
| **CAGR** | (Valor_Final / Valor_Inicial)^(1/n) - 1 | Crecimiento anual compuesto |
| **Análisis Vertical** | Partida / Base × 100 | Estructura porcentual |
| **Correlación** | Cov(X,Y) / (σ_X × σ_Y) | Relación entre variables |
| **R²** | 1 - (SS_res / SS_tot) | Calidad del ajuste lineal |

### 🎯 Framework de Análisis Temporal

1. **DIAGNÓSTICO** (¿Qué pasó?)
   - Análisis horizontal de evolución
   - Identificación de tendencias
   - Comparación vs benchmarks

2. **ANÁLISIS** (¿Por qué pasó?)
   - Correlaciones con factores macro
   - Análisis de drivers específicos
   - Impacto de decisiones estratégicas

3. **PROYECCIÓN** (¿Qué puede pasar?)
   - Extrapolación de tendencias
   - Análisis de escenarios
   - Bandas de confianza

4. **ACCIÓN** (¿Qué hacer?)
   - Estrategias por escenario
   - KPIs de monitoreo
   - Plan de contingencias

---

## 📝 Ejercicios Propuestos

### Ejercicio 1: Análisis de Ciclicalidad
Investiga la correlación entre el desempeño de cada empresa y:
- Ciclos económicos argentinos
- Precio del dólar
- Tasas de interés del BCRA
- Índices bursátiles (Merval)

### Ejercicio 2: Benchmarking Internacional
Compara nuestras empresas argentinas con:
- Líderes sectoriales latinoamericanos
- Empresas similares en mercados desarrollados
- ETFs sectoriales globales

¿Qué explica las diferencias? ¿Cómo pueden cerrar las brechas?

### Ejercicio 3: Modelo Predictivo Avanzado
Desarrolla un modelo más sofisticado que incluya:
- Variables macroeconómicas múltiples
- Efectos estacionales
- Variables dummy para eventos especiales
- Análisis de componentes principales

### Ejercicio 4: Dashboard Gerencial
Crea un dashboard ejecutivo que incluya:
- KPIs en tiempo real
- Alertas automáticas por desviaciones
- Comparación automática vs benchmarks
- Proyecciones actualizadas mensualmente

### Ejercicio 5: Análisis de Riesgo Temporal
Calcula para cada empresa:
- Value at Risk (VaR) del ROE
- Máxima caída histórica
- Tiempo de recuperación promedio
- Correlación con crisis sistémicas

---

## 🎓 Síntesis de la Unidad 2

### 🏆 Logros Alcanzados

A lo largo de la **Unidad 2** hemos desarrollado una comprensión integral del análisis de estados financieros:

#### **2.1 Fundamentos**
✅ Lectura e interpretación de estados financieros  
✅ Automatización con Python y pandas  
✅ Aplicación del marco normativo argentino  

#### **2.2 Análisis Operativo**
✅ Ratios de liquidez y gestión del capital de trabajo  
✅ Ratios de actividad y ciclo de conversión del efectivo  
✅ Benchmarking sectorial  

#### **2.3 Análisis Estratégico**
✅ Ratios de endeudamiento y estructura financiera  
✅ Ratios de rentabilidad y creación de valor  
✅ Análisis integral y ranking de empresas  

#### **2.4 Análisis Avanzado**
✅ Modelo DuPont y descomposición de la rentabilidad  
✅ Identificación de palancas de mejora  
✅ Análisis de sensibilidad y optimización  

#### **2.5 Perspectiva Temporal**
✅ Análisis horizontal y evolución temporal  
✅ Análisis vertical y cambios estructurales  
✅ Benchmarking dinámico y proyecciones  

### 🎯 Competencias Desarrolladas

Como futuros **Ingenieros Industriales**, ahora poseen:

**📊 Competencias Técnicas:**
- Dominio completo del análisis de ratios financieros
- Capacidad de automatización con Python
- Interpretación contextual para Argentina
- Análisis predictivo y de escenarios

**🎯 Competencias Estratégicas:**
- Identificación de drivers de rentabilidad
- Benchmarking sectorial y competitivo
- Desarrollo de recomendaciones accionables
- Comunicación efectiva con stakeholders

**🔮 Competencias Prospectivas:**
- Análisis de tendencias y proyecciones
- Gestión de incertidumbre y escenarios
- Adaptación a contextos macroeconómicos
- Innovación en metodologías de análisis

### 🚀 Aplicaciones Profesionales

Estas competencias son **directamente aplicables** en:

**🏭 Industria:**
- Evaluación de proveedores y socios estratégicos
- Justificación de proyectos de inversión
- Optimización de procesos con impacto financiero
- Gestión de capital de trabajo

**💼 Consultoría:**
- Due diligence para fusiones y adquisiciones
- Reestructuraciones empresariales
- Planes de mejora operativa
- Valuación de empresas

**🏦 Finanzas:**
- Análisis crediticio y rating
- Gestión de riesgos
- Investment banking
- Private equity y venture capital

**📈 Emprendimiento:**
- Monitoreo de performance de startups
- Preparación para rondas de inversión
- Benchmarking competitivo
- Planificación financiera estratégica

---

## 📚 Próximos Pasos: Unidad 3

En la **Unidad 3: Valuación de Activos** aplicaremos todo lo aprendido para:

🔹 **Valuación de Bonos**: Riesgo crediticio y duration  
🔹 **Valuación de Acciones**: Modelos de descuento de dividendos  
🔹 **Análisis Fundamental**: De ratios a valor intrínseco  
🔹 **Múltiplos Comparables**: Valuación relativa  
🔹 **Casos Argentinos**: CEDEARS y bonos soberanos  

### 🔗 Conexión con Análisis Financiero

Los ratios y análisis de esta unidad serán **inputs fundamentales** para:
- Determinar tasas de descuento apropiadas
- Proyectar flujos de fondos futuros
- Evaluar la calidad crediticia
- Comparar con empresas similares

---

## 🎖️ Reflexión Final

El **análisis de estados financieros** es mucho más que el cálculo mecánico de ratios. Es el arte de:

✨ **Contar la historia** detrás de los números  
✨ **Identificar oportunidades** de mejora y crecimiento  
✨ **Anticipar desafíos** y riesgos futuros  
✨ **Tomar decisiones** fundamentadas en evidencia  
✨ **Crear valor** para todos los stakeholders  

Como **Ingenieros Industriales** en Argentina, ustedes tienen la oportunidad única de:
- Optimizar procesos con impacto financiero medible
- Liderar transformaciones digitales en empresas tradicionales
- Desarrollar soluciones innovadoras para mercados emergentes
- Contribuir al crecimiento sostenible del país

**¡El futuro de la industria argentina está en sus manos!**

---

**📖 Recursos Adicionales:**
- [Financial Analysis - Coursera](https://www.coursera.org/specializations/financial-analysis)
- [Damodaran Online](http://pages.stern.nyu.edu/~adamodar/)
- [CFA Institute](https://www.cfainstitute.org/)
- [Mercado de Capitales Argentina](https://www.cnv.gov.ar/)

---

*Notebook desarrollado por Nicolás Félix para UTN La Plata*  
*Finanzas y Control Empresario - Ingeniería Industrial*  
*Julio 2025*

---

### 📞 Contacto y Consultas

**Profesor:** Nicolás Félix  
**Email:** [nicolas.felix@utn.edu.ar](mailto:nicolas.felix@utn.edu.ar)  
**Horarios de Consulta:** Martes y Jueves 14:00-16:00  
**Oficina:** Departamento de Ingeniería Industrial, UTN La Plata  

**Repositorio GitHub:** [finanzas-control-empresario-utn](https://github.com/nfelix23/finanzas-control-empresario-utn)  
**Issues y Consultas:** Utilizar el sistema de issues de GitHub para consultas técnicas  

---

*¡Gracias por su dedicación y compromiso con el aprendizaje!*  
*🇦🇷 UTN La Plata - Formando Ingenieros para el Futuro 🇦🇷*